In [1]:
import numpy as np
import pandas
import math
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from create_train_test_val_maps import *

In [2]:
train_map_revived = open_map('/home/cs231n/data/train')
val_map_revived = open_map('/home/cs231n/data/val')
test_map_revived = open_map('/home/cs231n/data/test')
#codes = [45021, 44004, 43004, 45008, 45002, 45007]

In [3]:
# get global index from (row, col) index
def sub2ind(array_shape, row, col):
    ind = row*array_shape[1] + col
    if row < 0 or row >= array_shape[0]:
        ind = -1
    if col < 0 or col >= array_shape[1]:
        ind = -1
    return ind

# get (row, col) index from global index
def ind2sub(array_shape, ind):
    row = int(ind) / array_shape[1]
    col = ind % array_shape[1]
    if ind < 0:
        row = -1
        col = -1
    if ind >=  array_shape[0]*array_shape[1]:
        row = -1
        col = -1
    return (row, col)

def softmax(x):
    """
    Compute softmax function for input. 
    Use tricks from previous assignment to avoid overflow
    """
	### YOUR CODE HERE
    xshift = np.max(x, axis = 1)
    xshift = xshift.reshape((x.shape[0],1))
    x = x - xshift
    s = np.exp(x) / np.sum(np.exp(x),axis = 1).reshape((x.shape[0],1))
	### END YOUR CODE
    return s

In [4]:
# Categorical Data to Numeric Format

# Train Data
category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','DTCID']
for vehicleID in train_map_revived.keys():
    for ATA6code in train_map_revived[vehicleID].keys():
        for time_window in range(len(train_map_revived[vehicleID][ATA6code].keys())):
            for i in range(len(train_map_revived[vehicleID][ATA6code][time_window])):
                for c in category_var:
                    train_map_revived[vehicleID][ATA6code][time_window][i][c] = train_map_revived[vehicleID][ATA6code][time_window][i][c].astype('category')
                    train_map_revived[vehicleID][ATA6code][time_window][i][c] = train_map_revived[vehicleID][ATA6code][time_window][i][c].cat.codes

# Validate Data
for vehicleID in val_map_revived.keys():
    for ATA6code in val_map_revived[vehicleID].keys():
        for time_window in range(len(val_map_revived[vehicleID][ATA6code].keys())):
            for i in range(len(val_map_revived[vehicleID][ATA6code][time_window])):
                for c in category_var:
                    val_map_revived[vehicleID][ATA6code][time_window][i][c] = val_map_revived[vehicleID][ATA6code][time_window][i][c].astype('category')
                    val_map_revived[vehicleID][ATA6code][time_window][i][c] = val_map_revived[vehicleID][ATA6code][time_window][i][c].cat.codes


# Test Data
for vehicleID in test_map_revived.keys():
    for ATA6code in test_map_revived[vehicleID].keys():
        for time_window in range(len(test_map_revived[vehicleID][ATA6code].keys())):
            for i in range(len(test_map_revived[vehicleID][ATA6code][time_window])):
                for c in category_var:
                    test_map_revived[vehicleID][ATA6code][time_window][i][c] = test_map_revived[vehicleID][ATA6code][time_window][i][c].astype('category')
                    test_map_revived[vehicleID][ATA6code][time_window][i][c] = test_map_revived[vehicleID][ATA6code][time_window][i][c].cat.codes


In [12]:
# TRAIN
num_states = 2
num_codes = len(codes)
num_time_windows = 10
num_iter = 100
models = {}

codes = []
code_window_map = {}
for vehicleID in train_map_revived.keys():
    for ATA6code in train_map_revived[vehicleID].keys():
        if ATA6code not in codes:
            codes.append(ATA6code)
        if ATA6code not in code_window_map:
            code_window_map[ATA6code] = {}
        for time_window in train_map_revived[vehicleID][ATA6code].keys():
            if time_window not in code_window_map[ATA6code]:
                code_window_map[ATA6code][time_window] = []
            for sequence_of_snapshots in train_map_revived[vehicleID][ATA6code][time_window]:
                code_window_map[ATA6code][time_window].append(sequence_of_snapshots)
                
for ATA6code in code_window_map.keys():
    for time_window in code_window_map[ATA6code].keys():
        lengths = []
        listofsequences = code_window_map[ATA6code][time_window]
        X = pandas.concat(listofsequences).as_matrix()[:,3:] 
        for sequence in listofsequences:
            lengths.append(sequence.shape[0]) 
        models[(ATA6code, time_window)] = hmm.GaussianHMM(n_components=num_states, n_iter=num_iter).fit(X,lengths)
        


In [14]:
# VALIDATE
lengths = []
for vehicleID in val_map_revived.keys():
    for ATA6code in val_map_revived[vehicleID].keys():
        for time_window in val_map_revived[vehicleID][ATA6code].keys():
            for sequence_of_snapshots in val_map_revived[vehicleID][ATA6code][time_window]:
                lengths.append(sequence_of_snapshots.shape[0])
num_val = sum(lengths)

sample = 0
labels = []
log_likelihoods = np.zeros((num_val, num_codes, num_time_windows))
for vehicleID in val_map_revived.keys():
    for ATA6code in val_map_revived[vehicleID].keys():
        for time_window in val_map_revived[vehicleID][ATA6code].keys():
            for sequence_of_snapshots in val_map_revived[vehicleID][ATA6code][time_window]:
                labels.append([codes.index(ATA6code),time_window])
                x = sequence_of_snapshots.as_matrix()[:,3:] 
                for pair in models.keys():
                    c = codes.index(pair[0])
                    w = pair[1]
                    print(sample,c,w)
                    log_likelihoods[sample,c,w] = models[pair].score(x)
        sample = sample+1                             

0 11 5
0 1 6
0 18 3
0 5 8
0 1 0
0 19 9
0 26 8
0 5 5
0 7 4
0 27 9
0 29 9
0 23 4
0 24 4
0 27 4
0 10 4
0 29 6
0 12 2
0 32 7
0 8 0
0 20 1
0 0 6
0 17 7
0 9 5
0 13 5
0 15 5
0 17 8
0 4 4
0 15 8
0 28 3
0 8 9
0 7 3
0 28 6
0 8 4
0 24 9
0 10 7
0 30 3
0 25 8
0 32 4
0 16 2
0 19 1
0 12 1
0 17 2
0 26 2
0 15 2
0 4 6
0 31 6
0 11 4
0 13 8
0 5 2
0 12 8
0 28 0
0 21 8
0 24 3
0 1 3
0 21 4
0 25 6
0 11 2
0 10 2
0 3 5
0 5 9
0 8 7
0 6 1
0 19 2
0 14 5
0 17 1
0 31 8
0 23 5
0 21 6
0 1 4
0 4 5
0 26 4
0 5 1
0 7 8
0 4 1
0 23 0
0 24 0
0 30 0
0 28 5
0 10 8
0 2 5
0 29 2
0 25 1
0 9 1
0 16 5
0 20 5
0 3 0
0 0 2
0 16 8
0 7 1
0 19 7
0 14 0
0 7 5
0 2 1
0 6 2
0 19 5
0 5 6
0 1 1
0 26 9


ValueError: rows of transmat_ must sum to 1.0 (got [ 1.  0.])

In [ ]:
#accuracy
errors = np.zeros((labels.shape))
correct_codes = np.zeros((labels.shape))
probabilities = np.zeros(log_likelihoods.shape)
time_counts = 0
for s in range(0,num_val):
    probabilities_temp = softmax(log_likelihoods[s,:,:].reshape(1,num_codes*num_time_steps))
    probabilities[s] = probabilities_temp.reshape(num_codes, num_time_steps) 
    max_index = np.unravel_index(probabilities[s].argmax(), probabilities[s].shape)
    errors[s] = max_index - np.array([codes.index(labels[s,0]),labels[s,1]])
    correct_codes[s] = max_index == np.array([codes.index(labels[s,0]),labels[s,1]])
    if correct_codes[s,0] == True and correct_codes[s,1] == True:
        time_counts = time_counts + 1        
error_sum = sum(errors)
correct_sum = sum(correct_codes)
code_accuracy = correct_sum[0]/num_val
k_accuracy = time_counts/num_val
mse = (1.0/num_val)*sum(np.power(errors[:,0],2))


print('ATA6: Mean Squared Error',mse)
print('ATA6: Accuracy', code_accuracy)
print('K: Accuracy', k_accuracy)


In [ ]:
code_probabilities = np.zeros((num_val, num_codes))
predicted_codes = np.zeros(num_val)
for s in range(0,num_val):
    probabilities_temp = softmax(log_likelihoods[s,:,:].reshape(1,num_codes*num_time_steps))
    code_probabilities[s,:] = np.sum(probabilities_temp.reshape(num_codes, num_time_steps),axis=1)
    predicted_codes[s] = np.argmax(code_probabilities[s,:])
code_accuracy = np.sum(predicted_codes == labels[:,0])/num_val
print("Marginalized Predicted Code Accuracy: ", code_accuracy)

In [ ]:
codes = [1,2,3]

In [ ]:
(1 not in codes)